# Learning Goal: demonstrating "long-term" selection

The mutation rate possessed by a single organism does not affect its fitness.

That is, there is selection *on the organism's external phenotype*, but because mutation rate *only affects the offspring*, is not "visible" to single-generation selection. 

Nonetheless, this lab will demonstrate that there is “long-term selection” on mutation rate because it produces differential survival of clades.

# Setup
### Six possible genotypes

We will consider a fixed population size of N=10<sup>9</sup> individual organisms, each of which has a certain genotype.

There are two possible alleles at the "selected locus": *A* and *B*. The alleles *A* and *B* will have different fitnesses.

There are three possible alleles at the "mutator locus": *M<sub>2</sub>*, *M<sub>3</sub>*, *M<sub>4</sub>*. There is no direct selection on the mutator locus; it just sets the rate of mutation from *A*->*B* or *B*->*A* in offspring.

This makes a total of 2 * 3 = 6 possible genotypes:

|                   | A                       | B |
| :--------------   | :------:                | :----:               |
| *M<sub>2</sub>*   |   (*M<sub>2</sub>*, A)  | (*M<sub>2</sub>*, B) |
| *M<sub>3</sub>*   |   (*M<sub>3</sub>*, A)  | (*M<sub>3</sub>*, B) |
| *M<sub>4</sub>*   |   (*M<sub>4</sub>*, A)  | (*M<sub>4</sub>*, B) |

The organisms in our model are asexual - there is no mating/recombination. Offspring have a single parent. 

### Each generation of evolution
At each generation, in a typical population genetics model (like this one), we will:

1. Apply **selection** to determine the *number of offspring* that each individual organism will have.
2. Apply **mutation** to the selected allele at a certain mutation rate.

If there's no mutation, the offspring inherits the parent's value of A or B. If there is a mutation, the inherited allele switches *A->B* or *B->A* from the parent's value.

### Selection
There will be two environments in our model. At any generation the current "environmental state” is either *A* or *B*. This state will fluctuate over time on a schedule that I will explain below.

Environment *A* favors selected allele *A*, and environment *B* favors selected allele *B*.

Organisms holding the allele at the selected locus that matches the state of the environment will produce more offspring on average: they are more fit.

Specifically, given a “selection coefficient” *s*, the matching (fitter) organisms produce 1+*s* times as many offspring. This means that the fitter organisms will increase in relative number.

### Mutation
The alleles M2, M3, M4 at the mutator locus produce mutation rates in the selected locus of 10-2, 10-3, 10-4, respectively. The rate is symmetric, A->B, or B->A.

For example, at each generation, an individual with genotype (A, M2) produces offspring that are 99% (A, M2), and 1% (B, M2); that is, the A mutates to B in 1 out of 102 offspring.

Similarly, an individual with genotype (A, M4) produces offspring that are 99.99% (A, M4), and 0.01% (B, M4);  that is, the A mutates to B in 1 out of 104 offspring.

It's symmetrical in the other direction from B->A.

The mutator locus itself never mutates. That means that all descendants of an individual possessing an M2 at the mutator locus will always possess an M2 (and likewise for M3, M4). This constraint in our particular experimental setup makes the mutator locus a convenient marker of clade identity. So, we can refer to the M2 clade as being founded by all the M2 individuals at generation 0, and all of its descendant members will always possess an M2 forever.


# Learning Goal – elaborated:

In each generation,
- Selection determines the *number of offspring* an organism has – its fitness.
- However, selection does not determine the *genotypes of the offspring*.
- Rather, *the mutation rate* determines the *distribution of genotypes* in the offspring.

So, individual organisms are selected (i.e., their fitness is determined) depending *only* on the allele they hold at the selected locus, and whether it matches the environmental state.

The mutation rate is not "visible" to selection over one generation.

However – by affecting the distribution of genotypes in the offspring – the mutation rate becomes visible to multi-generation selection!

Recall that a clade will "survive" (or not) if it retains one member (or not). That depends on the distribution of the offspring genotypes.

That is, clades can experience "long-term" selection on mutation rate!

- Moreover, this is just a special case of a more general selection on clades for not just mutation rate, but *for anything that affects the distribution of genotypes in their members*. (but we don't look at that in this demonstration)

- More-moreover, in the even longer-term, there is selection on *change* in mutation rate... and the change in *that*... and the change in *that*! That is, there is a recursive dynamic! (but not in this demonstration because we fix the mutation rates)


---
---
# Simulations

In [ ]:
from importlib import reload
import evosim
reload(evosim)
display("default arguments:", evosim.default_args)

---
---
### Drift (no selection or mutation)

Drift is change in genotype frequencies due to stochasticity (e.g., in number of offspring produces

Drift is stronger with low N (population size).

- Change N between 1e2 and 1e9

Re-run (Ctrl-return) multiple times with different parameters.

In [ ]:
# Goal: to see effect of population size on drift
override_args = evosim.default_args.copy()

override_args.s = 0.0 # no selection when s=0.0
override_args.skipMutation = True # True = no mutation
override_args.N = 1e3 # 1e3
override_args.T = 4e3 # 3e3
override_args.numClades = 1 # 1
#override_args.print_stats = True

evosim.evosim(override_args)
# Re-run multiple times (Ctrl-return) with different parameters.
# experiment 1: change N between 1e2 and 1e9, re-run multple times (Ctrl-return) 
# experiment 2: compare large N to N=-1 (which means infinite population)
# experiment 3: restore N=1e4 and T=2e3. Increase numClades to 2 or 3

---
---
### Selection (without mutation)

Change N to see differential effect of selection vs drift.

Change s to see effect of selection strength.

- if sT is low (<= 5.0), then fixation won't usually happen in one epoch (T generations)
- if Ns is low (<= 1.0), then drift will be strong relative to selection
- Type *A* should fix if Ns > 1, and sT > 5


Re-run multiple times (Ctrl-return) with different parameters.

In [ ]:
# Goal: to see effect of selection vs. drift
override_args = evosim.default_args.copy()

override_args.s = 0.1 # no selection when s=0.0
override_args.skipMutation = True # True = no mutation

override_args.N = 1e5
override_args.numClades = 3
override_args.T = 1e3

evosim.evosim(override_args)

# Re-run multiple times (Ctrl-return) with different parameters.
# experiment 1: change s between 0.001, 0.001, 0.1, re-run multple times (Ctrl-return) 
# experiment 2: compare large N to N=-1 (which means infinite population)

---
---
### Selection vs mutation, and mutation-selection balance

In [ ]:
# Goal: to see effect of selection vs mutation, and mutation-selection balance
override_args = evosim.default_args.copy()

override_args.s = 0.01 # no selection when s=0.0
override_args.skipMutation = False # False = with mutation

override_args.minMu = 8

override_args.N = -1
override_args.numClades = 1
override_args.T = 1e3

evosim.evosim(override_args)

# Re-run multiple times (Ctrl-return) with different parameters.
# experiment 1: change minMu between 2 and 8, re-run multple times (Ctrl-return)
# experiment 2: compare large N to N=-1 (which means infinite population)

---
---
### Selection in alternating environments

In [ ]:
override_args = evosim.default_args.copy()

override_args.minMu = 2
override_args.numClades = 1

override_args.s = 0.1 # change this to see effect of selection strength
override_args.skipMutation = False

override_args.T = 1e3
override_args.numEpochs = 4
override_args.print_each_epoch = False

override_args.N = -1 # change this and re-run to see differential effect of selection vs drift
override_args.plotAB = True

evosim.evosim(override_args)

---
---
### Selection on different mutators (!!) in alternating environments

In [ ]:
override_args = evosim.default_args.copy()

override_args.minMu = 2
override_args.numClades = 3

override_args.s = 0.1 # change this to see effect of selection strength
override_args.skipMutation = False

override_args.T = 1e3
override_args.numEpochs = 4
override_args.print_each_epoch = False

override_args.N = -1 # change this and re-run to see differential effect of selection vs drift
override_args.plotAB = True

evosim.evosim(override_args)